### 딕셔너리에 존재하지 않는 키를 조회할 때

### 1. `setdefault()`

In [3]:
sentences = [
    ['ones', 'upon', 'a time'], 
    ['there', 'were', 'three', 'bears'], 
    ['in', 'the middle', 'of', 'forest.'],
    ['there', 'was', 'a', 'girl', 'too'], 
    ['in', 'the', 'tower', 'of', 'castle'],
    ['next', 'to', 'the', 'forest.']
]
index = {}
for line_no, sen in enumerate(sentences):
    for word_no, word in enumerate(sen):
        occur = index.get(word, [])
        occur.append(f'{line_no}:{word_no}')
        index[word] = occur
        
for item in index.items():
    print(item)

('ones', ['0:0'])
('upon', ['0:1'])
('a time', ['0:2'])
('there', ['1:0', '3:0'])
('were', ['1:1'])
('three', ['1:2'])
('bears', ['1:3'])
('in', ['2:0', '4:0'])
('the middle', ['2:1'])
('of', ['2:2', '4:3'])
('forest.', ['2:3', '5:3'])
('was', ['3:1'])
('a', ['3:2'])
('girl', ['3:3'])
('too', ['3:4'])
('the', ['4:1', '5:2'])
('tower', ['4:2'])
('castle', ['4:4'])
('next', ['5:0'])
('to', ['5:1'])


중첩 for문에서 단어 발생을 처리하는 코드 세 줄은 `dict`의 `setdefault()`를 사용하면 한 줄로 바꿀 수 있다.

In [4]:
index = {}

for line_no, sen in enumerate(sentences):
    for word_no, word in enumerate(sen):
        index.setdefault(word, []).append(f'{line_no}:{word_no}')
        
for item in index.items():
    print(item)

('ones', ['0:0'])
('upon', ['0:1'])
('a time', ['0:2'])
('there', ['1:0', '3:0'])
('were', ['1:1'])
('three', ['1:2'])
('bears', ['1:3'])
('in', ['2:0', '4:0'])
('the middle', ['2:1'])
('of', ['2:2', '4:3'])
('forest.', ['2:3', '5:3'])
('was', ['3:1'])
('a', ['3:2'])
('girl', ['3:3'])
('too', ['3:4'])
('the', ['4:1', '5:2'])
('tower', ['4:2'])
('castle', ['4:4'])
('next', ['5:0'])
('to', ['5:1'])


`setdefault(key[, default])`는 딕셔너리에 `key`가 없을 경우 (`key`, `default`) 쌍을 딕셔너리에 추가한 후 `default`를 리턴한다. 위의 경우 `word`를 키로 가지고 있지 않을 때 word에 해당하는 값을 키로, 빈 리스트가 값인 쌍을 딕셔너리에 새로 추가한다. 이후 word에 해당하는 키를 조회할 때 리스트를 불러오고, 불러온 리스트에 문자열을 덧붙인다. 

### 2. defaultdict

`defaultdict`은 `__getitem__()`으로 존재하지 않는 키를 
조회할 때마다 기본값을 생성하기 위한 callable을 제공한다. 기본값을 생성하는 callable은 `defaultdict`의 `default_factory`라는 속성에 저장된다.

In [5]:
from collections import defaultdict

index = defaultdict(list)

for line_no, sen in enumerate(sentences):
    for word_no, word in enumerate(sen):
        index[word].append(f'{line_no}:{word_no}')
        
for item in index.items():
    print(item)

('ones', ['0:0'])
('upon', ['0:1'])
('a time', ['0:2'])
('there', ['1:0', '3:0'])
('were', ['1:1'])
('three', ['1:2'])
('bears', ['1:3'])
('in', ['2:0', '4:0'])
('the middle', ['2:1'])
('of', ['2:2', '4:3'])
('forest.', ['2:3', '5:3'])
('was', ['3:1'])
('a', ['3:2'])
('girl', ['3:3'])
('too', ['3:4'])
('the', ['4:1', '5:2'])
('tower', ['4:2'])
('castle', ['4:4'])
('next', ['5:0'])
('to', ['5:1'])


`index[word]`는 `__getitem__()`을 호출한다. `__getitem__()`에 의해 `word`가 `index`에 없다는 게 확인되면, `index`의 `default_factory`속성에 저장된 callable을 호출해서 없는 값에 대한 항목을 생성한다. `default_factory`에는 `list`클래스의 생성자가 저장되어 있으므로
빈 리스트가 `index[word]`에 할당된다. `__getitem__()`은 생성된 리스트를 리턴하므로 `append()`호출도 가능하다.

`default_factory`에 `int`를 저장하면 아래와 같이 응용할 수 있다. `int`생성자의 기본값은 0이다.

In [11]:
s = 'mississippi'
d = defaultdict(int)
for k in s:
    d[k] += 1
    
sorted(d.items())

[('i', 4), ('m', 1), ('p', 2), ('s', 4)]

### 3. `UserDict`

UserDict는 dict를 상속하지 않고 내부에 실제 항목을 갖고 있는 data라는 dict객체를 갖고 있다.
이렇게 구현하므로서 __setitem__ 등의 특수메서드를 구현할 때 발생하는 원치않는 재귀호출을 피할 수 있으며
__contains__메서드를 간단하게 구현할 수 있다,

In [9]:
from collections import UserDict

class IntToStrKeyDict(UserDict):
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def __contains__(self, key):
        return str(key) in self.data
    
    def __setitem__(self, key, item):
        self.data[str(key)] = item
        
userdict = IntToStrKeyDict({'1': 'red', '2': 'orange', '3': 'yellow'})
print(userdict[1])
print(userdict[2])
print(userdict[3])
print(userdict['red'])

red
orange
yellow


KeyError: 'red'